In [1]:
%load_ext autoreload
%load_ext tensorboard
%autoreload 2

In [2]:
# System imports
import os
import sys
import yaml

# External imports
import scipy as sp
import numpy as np
import pandas as pd
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

sys.path.append('..')
device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import nersc_tensorboard_helper

In [3]:
from LightningModules.GNN.Models.ecgnn import ECGNN

In [4]:
with open("example_gnn.yaml") as f:
        hparams = yaml.load(f, Loader=yaml.FullLoader)

In [5]:
model = ECGNN(hparams)
logger = TensorBoardLogger("tb_ecgnn_logs",name = "ecgnn")
trainer = Trainer(gpus=1,max_epochs=20)
torch.autograd.set_detect_anomaly(True)
trainer.fit(model)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name           | Type        | Params
-----------------------------------------------
0 | input_network  | Sequential  | 384   
1 | node_network   | NodeNetwork | 17.2 K
2 | edge_network   | EdgeNetwork | 18.6 K
3 | output_network | Sequential  | 12.7 K
-----------------------------------------------
49.0 K    Trainable params
0         Non-trainable params
49.0 K    Total params


Epoch 0:   0%|          | 0/40 [00:00<?, ?it/s]                       1979 15198
0
3
Epoch 0:   0%|          | 0/40 [00:02<?, ?it/s]


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.cuda.FloatTensor [18123, 1]], which is output 0 of UnsqueezeBackward0, is at version 3; expected version 0 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!

In [17]:
test_results = trainer.test(ckpt_path=None)
print(test_results)

Set SLURM handle signals.


Testing: 100%|██████████| 20/20 [00:09<00:00,  2.22it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'current_lr': 0.004799999762326479,
 'eff': 0.6384192109107971,
 'loss': 33.44152069091797,
 'pur': 0.6111376285552979,
 'ratio': 0.22399879327249417,
 'truth': array([0., 1., 0., ..., 0., 1., 1.], dtype=float32),
 'val_loss': 36.890316009521484}
--------------------------------------------------------------------------------
[{'loss': 33.44152069091797, 'eff': 0.6384192109107971, 'truth': array([0., 1., 0., ..., 0., 1., 1.], dtype=float32), 'ratio': 0.22399879327249417, 'val_loss': 36.890316009521484, 'pur': 0.6111376285552979, 'current_lr': 0.004799999762326479}]


In [15]:
os.environ['TENSORBOARD_BINARY'] = "/global/homes/j/jferguso/.conda/envs/exatrkx-tracking/bin/tensorboard"
%tensorboard --logdir tb_ecgnn_logs/ --port 0
nersc_tensorboard_helper.tb_address()


In [11]:
!ps aux | grep tensorboard

jferguso 16183 21.9  0.0  14136  3556 pts/2    Ss+  15:53   0:00 /bin/bash -c ps aux | grep tensorboard
jferguso 16193  0.0  0.0   8984   900 pts/2    S+   15:53   0:00 grep tensorboard
